In [ ]:
import yfinance as yf
import pandas as pd
import requests


## 200d-EMA & 52 Week High


In [11]:
# Define the stock universe
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# Function to get 52-week high
def get_52_week_high(symbol):
    try:
        df = yf.download(symbol, period="1y")  # Fetch last 1 year of data
        return float(df["High"].max())  # Extract float value
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")
        return None

# Function to get 200-day EMA for a stock
def get_200d_ema(symbol):
    try:
        df = yf.download(symbol, period="1y")  # Get 1 year of data
        df["200d_EMA"] = df["Close"].ewm(span=200, adjust=False).mean()
        return df["200d_EMA"].iloc[-1]  # Return latest EMA value
    except Exception as e:
        print(f"Error fetching {symbol}: {e}")
        return None

# Store results in a list
ema_data = []

# Loop through each sector and stock
for sector, stocks in us_stock_universe.items():
    for stock in stocks:
        high_value = get_52_week_high(stock)
        ema_value = get_200d_ema(stock)
        ema_data.append({"Sector": sector, "Stock": stock, "200d EMA": ema_value,"52-Week High": high_value})

# Convert results to a DataFrame
ema_df = pd.DataFrame(ema_data)

# Print results
ema_df.to_csv("200dema.csv", index=False)


[*********************100%***********************]  1 of 1 completed
C:\Users\kshit\AppData\Local\Temp\ipykernel_38892\706131671.py:19: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(df["High"].max())  # Extract float value
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
C:\Users\kshit\AppData\Local\Temp\ipykernel_38892\706131671.py:19: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  return float(df["High"].max())  # Extract float value
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
C:\Users\kshit\AppData\Local\Temp\ipykernel_38892\706131671.py:19: FutureWarning: Calling float on a single element Series 

## Alpha & Beta


In [ ]:
# Define the stock universe
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# Flatten stock universe to get all tickers
stocks = [ticker for sector in us_stock_universe.values() for ticker in sector]

# Define benchmark (S&P 500)
benchmark_ticker = "^GSPC"

# Define period for historical data
period = "5y"

# Approximate risk-free rate (4% annually)
risk_free_rate = 0.04

# Fetch benchmark data
benchmark_data = yf.Ticker(benchmark_ticker).history(period=period)["Close"]
benchmark_return = (benchmark_data.iloc[-1] / benchmark_data.iloc[0]) - 1  # Total return over the period

# Store Alpha values
alpha_data = []

for stock in stocks:
    try:
        stock_data = yf.Ticker(stock).history(period=period)["Close"]
        stock_return = (stock_data.iloc[-1] / stock_data.iloc[0]) - 1  # Total return over the period

        # Fetch Beta directly from Yahoo Finance
        beta = yf.Ticker(stock).info.get("beta", None)

        if beta is not None:
            # Calculate Alpha
            alpha = (stock_return - risk_free_rate) - (beta * (benchmark_return - risk_free_rate))

            # Store data
            alpha_data.append({"Ticker": stock, "Alpha (Annualized)": alpha / 5,"Beta": beta})  # Convert to annualized Alpha

    except Exception as e:
        print(f"Error fetching Alpha for {stock}: {e}")

# Save Alpha values
alpha_df = pd.DataFrame(alpha_data)
alpha_df.to_csv("alpha.csv", index=False)

print("✅ Alpha values saved to alpha.csv")


## Liquidity

In [ ]:
# Define the stock universe
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# Flatten stock universe to get all tickers
stocks = [ticker for sector in us_stock_universe.values() for ticker in sector]

# Define the period for fetching average volume (e.g., 30 days)
period = "30d"

# Initialize a list to store liquidity data
liquidity_data = []

for stock in stocks:
    try:
        ticker = yf.Ticker(stock)
        hist = ticker.history(period=period)
        info = ticker.info
        
        if not hist.empty:
            avg_volume = hist['Volume'].mean()
            market_cap = info.get("marketCap", None)
            bid = info.get("bid", None)
            ask = info.get("ask", None)
            last_close = hist['Close'].iloc[-1]
            
            # Calculate bid-ask spread percentage
            if bid and ask and bid > 0:
                bid_ask_spread = ((ask - bid) / ((ask + bid) / 2)) * 100
            else:
                bid_ask_spread = None
            
            # Calculate turnover ratio
            if market_cap and market_cap > 0:
                turnover_ratio = avg_volume / market_cap
            else:
                turnover_ratio = None
            
            # Calculate Amihud's Illiquidity Ratio
            daily_return = hist['Close'].pct_change().abs()
            dollar_volume = hist['Volume'] * hist['Close']
            if not dollar_volume.empty:
                amihud_ratio = (daily_return / dollar_volume).mean()
            else:
                amihud_ratio = None
            
            liquidity_data.append({
                "Ticker": stock,
                "Bid-Ask Spread (%)": bid_ask_spread,
                "Turnover Ratio": turnover_ratio,
                "Amihud's Illiquidity Ratio": amihud_ratio
            })
    except Exception as e:
        print(f"Error fetching data for {stock}: {e}")

# Convert to DataFrame
df = pd.DataFrame(liquidity_data)

# Save to CSV
df.to_csv("liquidity.csv", index=False)

print("Liquidity data saved to stock_liquidity.csv")


## Intrinsic Value

In [ ]:
# Define the stock universe
us_stock_universe = {
    "Technology": ["AAPL", "MSFT", "GOOGL", "NVDA", "META"],
    "Banking & Financials": ["JPM", "BAC", "WFC", "C", "GS"],
    "Consumer Goods": ["PG", "KO", "PEP", "MO", "MNST"],
    "Pharma & Healthcare": ["JNJ", "PFE", "MRNA", "LLY", "UNH"],
    "Energy": ["XOM", "CVX", "COP", "PSX", "SLB"],
    "Automobile": ["TSLA", "F", "GM", "RIVN", "LCID"],
    "Retail & E-commerce": ["AMZN", "WMT", "COST", "TGT", "HD"],
    "Aerospace & Defense": ["BA", "LMT", "NOC", "GD", "RTX"],
    "Telecom & Media": ["VZ", "T", "DIS", "NFLX", "CMCSA"],
    "Semiconductors": ["AMD", "TSM", "INTC", "AVGO", "QCOM"]
}

# Flatten stock universe to get all tickers
stocks = [ticker for sector in us_stock_universe.values() for ticker in sector]

# Initialize list to store intrinsic values
stock_data_list = []

def calculate_intrinsic_value(ticker):
    try:
        stock = yf.Ticker(ticker)
        eps = stock.info.get("trailingEps", None) or stock.info.get("forwardEps", None)
        growth_rate = stock.info.get("earningsGrowth", None)
        pe_ratio = stock.info.get("trailingPE", None) or stock.info.get("forwardPE", None)
        
        # Skip if EPS or P/E ratio is missing or non-positive
        if eps is None or eps <= 0 or pe_ratio is None or pe_ratio <= 0:
            return None
        
        # Use a default growth rate if missing
        if growth_rate is None or growth_rate <= 0:
            growth_rate = 0.05  # Default 5%
        
        # Clamp growth rate between 0% and 15%
        growth_rate = max(0.00, min(growth_rate, 0.15))
        
        # Calculate intrinsic value using the formula: EPS * (1 + r) * P/E Ratio
        intrinsic_value = eps * (1 + growth_rate) * pe_ratio
        
        return round(intrinsic_value, 2)
    except Exception as e:
        print(f"Error calculating intrinsic value for {ticker}: {e}")
    return None

for stock in stocks:
    intrinsic_value = calculate_intrinsic_value(stock)
    stock_data_list.append({
        "Ticker": stock,
        "Intrinsic Value": intrinsic_value
    })

# Convert to DataFrame
df = pd.DataFrame(stock_data_list)

# Save to CSV
df.to_csv("intrinsic_value.csv", index=False)

print("Intrinsic Value saved to stock_intrinsic_value.csv")


## Macroeconomic Factors

In [ ]:
# API Key (Replace with your actual FRED API key)
FRED_API_KEY = "c0c8c88711d864a221baf691fb4680ff"

# Macroeconomic indicators to fetch
FRED_INDICATORS = {
    "GDP": "GDP",  # US Gross Domestic Product
    "Interest Rate": "DFF",  # Federal Funds Rate
}

# Function to fetch macroeconomic data from FRED
def fetch_fred_data(indicator_code):
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={indicator_code}&api_key={FRED_API_KEY}&file_type=json"
    response = requests.get(url).json()

    if "observations" not in response:
        print(f"⚠️ Error fetching {indicator_code}: {response}")
        return None

    # Convert response to DataFrame
    data = pd.DataFrame(response["observations"])
    data["date"] = pd.to_datetime(data["date"])
    data.set_index("date", inplace=True)

    # Ensure 'value' column is numeric, replacing non-numeric values
    data["value"] = pd.to_numeric(data["value"], errors="coerce")  # Convert to float, replace errors with NaN

    return data.dropna()  # Remove rows with NaN values

# Fetch all macroeconomic indicators
macro_data = {name: fetch_fred_data(code) for name, code in FRED_INDICATORS.items()}

# Function to calculate required averages
def calculate_and_save_gdp(df, name, filename):
    if df is None or df.empty:
        print(f"⚠️ No data available for {name}.")
        return

    # Print available years for debugging
    available_years = df.index.year.unique()
    print(f"✅ Available years for {name}: {available_years}")

    # Define the most recent year available
    last_available_year = df.index.year.max()

    # Previous 2 years' average
    previous_two_years = [last_available_year - 1, last_available_year - 2]
    prev_2_years_avg = df[df.index.year.isin(previous_two_years)]["value"].mean()

    # Fetch monthly data for the last available year
    current_year_data = df[df.index.year == last_available_year]
    if current_year_data.empty:
        print(f"⚠️ No data available for {name} in {last_available_year}. Using previous year instead.")
        last_available_year -= 1
        current_year_data = df[df.index.year == last_available_year]
    
    # Assign month indices starting from 1
    current_year_data = current_year_data.sort_index()
    current_year_data["Month_Index"] = range(1, len(current_year_data) + 1)

    # Create a DataFrame to store results
    results_df = pd.DataFrame({
        "Metric": ["Previous 2 Years Avg"] + list(current_year_data["Month_Index"]),
        "Value": [prev_2_years_avg] + list(current_year_data["value"])
    })

    # Save results to CSV
    results_df.to_csv(filename, index=False)
    print(f"✅ {name} data saved to {filename}")

def calculate_and_save_interest(df, name, filename):
    if df is None or df.empty:
        print(f"⚠️ No data available for {name}.")
        return

    # Print available years for debugging
    available_years = df.index.year.unique()
    print(f"✅ Available years for {name}: {available_years}")

    # Define the most recent year available
    last_available_year = df.index.year.max()

    # Previous 2 years' average
    previous_two_years = [last_available_year - 1, last_available_year - 2]
    prev_2_years_avg = df[df.index.year.isin(previous_two_years)]["value"].mean()

    # Current year (or last available year) average
    current_year_data = df[df.index.year == last_available_year]

    if current_year_data.empty:
        print(f"⚠️ No data available for {name} in {last_available_year}. Using previous year instead.")
        last_available_year -= 1
        current_year_data = df[df.index.year == last_available_year]

    # Calculate overall average for the current year
    current_year_avg = current_year_data["value"].mean()

    # Create a DataFrame to store results
    results_df = pd.DataFrame({
        "Metric": ["Previous 2 Years Avg", "Current Year Avg"],
        "Value": [prev_2_years_avg, current_year_avg]  # Store only one "Current Year Avg"
    })

    # Save results to CSV
    results_df.to_csv(filename, index=False)
    print(f"✅ {name} data saved to {filename}")

# Calculate and save for GDP and Interest Rate
calculate_and_save_gdp(macro_data["GDP"], "GDP", "gdp.csv")
calculate_and_save_interest(macro_data["Interest Rate"], "Interest Rate", "interest_rate.csv")


## Analyzing GDP and Interest

In [ ]:
gdp_data=pd.read_csv("gdp.csv")
interest_data=pd.read_csv("interest_rate.csv")

gdp_flag=1
absolute_flag=0

gdp=(gdp_data["Value"])
prev_avg_gdp=gdp[0]
current_avg=0
for i in range(1,len(gdp)):
    current_avg+=gdp[i]
current_avg/=(len(gdp)-1)
current_avg

interest=(interest_data["Value"])

if(prev_avg_gdp<current_avg):
    for i in range(1,len(gdp)-1):
        if(gdp[i+1]-gdp[i]<0):
            gdp_flag=0
            break
else:
    gdp_flag=0

if(interest[1]>interest[0]) or (interest[1]>=3):
    absolute_flag=1

result_df = pd.DataFrame({
    "Metric":["GDP Flag", "Absolute Flag"],
    "Value": [gdp_flag,absolute_flag]
})

result_df.to_csv("Flags.csv",index=False)
print("Flags stored in Flags.csv\n")